## Install depedencies

In [ ]:
# Ignore any "Restart session" suggestions
!pip install rasa[spacy]
!python  -m spacy download en_core_web_md
!pip install nest-asyncio

## You must drag and drop all the Rasa-related files in the colab Files on your left.

First, copy:
- endpoints.yml
- domain.yml
- credentials.yml
- config.yml

Then, create 2 folders: `actions` and `data`.

In `actions` copy:
- actions.py

In `data` copy:
- nlu.yml
- stories.yml
- rules.yml


## Train the Virtual Assistant model

For each different model, delete all old Rasa files and drag-and-drop the new ones.

In [ ]:
# Train the Virtual Assistant model
!rasa train

## Deploy the assistant

In [ ]:
# Setup the environment
import nest_asyncio
import threading
import os
import rasa

nest_asyncio.apply()

# Function to run Rasa server
def run_rasa():
    os.system('rasa run -m models --enable-api --cors "*" --debug')

# Function to run Rasa action server
def run_action_server():
    os.system('rasa run actions --debug')

# Create threads for Rasa server and action server
rasa_thread = threading.Thread(target=run_rasa)
action_thread = threading.Thread(target=run_action_server)

# Start the threads
rasa_thread.start()
action_thread.start()


## Chat with your model

In [ ]:
import requests

# Interactive loop to prompt user for input and print Rasa responses
def interactive_chat():
    rasa_url = "http://localhost:5005/webhooks/rest/webhook"
    while True:
        message = input("You: ")
        if message.lower() == "exit":
            print("Exiting chat...")
            break
        payload = {"sender": "user", "message": message}
        response = requests.post(rasa_url, json=payload)
        for res in response.json():
            print(f"Bot: {res['text']}")

# Start the interactive chat and chat with your model
interactive_chat()
